In [1]:
# pip3 install google-cloud-bigquery google-cloud-bigquery-storage pyarrow tqdm ipywidgets pandas matplotlib db-dtypes

In [2]:
from google.cloud import bigquery

In [3]:
bq = bigquery.Client()

In [5]:
for ds in bq.list_datasets("cs320-f21"):
    print(ds.dataset_id)

apr24
apr25
loans1
p7
test2


In [10]:
# NAME:
# project.dataset.table
dataset = bigquery.Dataset("cs320-f21.apr26")
# dataset.location = ...
bq.create_dataset(dataset, exists_ok=True)

Dataset(DatasetReference('cs320-f21', 'apr26'))